<center><h1> XTTS-v2: Single-Speaker Fine-Tuning </center></h1>
<center> Roberto Caamano, Giuseppe Di Roberto </center>

# <img src="img/XTTS.png" width=600 height=500 />

In [ ]:
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import torch
import torchaudio
import os
import re

In [ ]:
# Set up file paths
DATASET = "noramlized_personal_voice"


BPE_path = "XTTS-files/vocab.json"

checkpoint_dir = f"training_outputs/xttsv2_finetune_20250504_1250-May-04-2025_12+50PM-ca1939c"
config_path =f"{model_path}/config.json"

speaker_ref = f"datasets/{DATASET}/wavs/chunk_0016.wav"


In [ ]:
# Define input text
text = "Hello world, I can speak."

# Init Xtts and load config object
cfg = XttsConfig()
cfg.load_json(config_path)


# Init model and load from checkpoint
model = Xtts.init_from_config(cfg)

model.load_checkpoint(
    cfg,
    checkpoint_dir=checkpoint_dir,
    vocab_path=vocab_path,
    eval=True, 
)

# Set to eval
model.to(device).eval()

# Get the gpt conditonal latent codes and speaker_encoder from the reference audio mel spectrogram
gpt_cond_latent, speaker_encoder = model.get_conditioning_latents(
    audio_path=[speaker_ref], # Speaker reference wav pointed here. Multiple can be used. Important for quality of output
    gpt_cond_len=cfg.gpt_cond_len, #  Context window size of latents being passed to GPT 
    gpt_cond_chunk_len=cfg.gpt_cond_chunk_len, # How many chunks audio tokens split into before going to PercieverResampler
    max_ref_length=cfg.max_ref_len, # Limits how much of the speaker reference audio is used.
)


# Model's inference method
output = model.inference(
            text=text, # Input text
            language="en", # Set language to english 
            gpt_cond_latent=gpt_cond_latent, # Pass conditional latents to GPT
            speaker_embedding=speaker_encoder, # Pass Speaker Encoder to Decoder  
            temperature=0.75,
            speed=1,
            length_penalty=cfg.length_penalty,
            repetition_penalty=cfg.repetition_penalty,
            top_k=cfg.top_k,
            top_p=cfg.top_p,
        )

# Create wav tensor
wav_tensor = torch.tensor(out["wav"]).unsqueeze(0)  # shape: (1, samples)


In [ ]:
'''
Notes from TTS.tts.models.xtts

gpt_codes = self.gpt.generate(
                    cond_latents=gpt_cond_latent,
                    text_inputs=text_tokens,
                    input_tokens=None,
                    do_sample=do_sample,
                    top_p=top_p,
                    top_k=top_k,
                    temperature=temperature,
                    num_return_sequences=self.gpt_batch_size,
                    num_beams=num_beams,
                    length_penalty=length_penalty,
                    repetition_penalty=repetition_penalty,
                    output_attentions=False,
                    **hf_generate_kwargs,
                )


gpt_latents = self.gpt(
    text_tokens,
    text_len,
    gpt_codes,
    expected_output_len,
    cond_latents=gpt_cond_latent,
    return_attentions=False,
    return_latent=True,
)
                
'''